## Load RMI Utilities Transition Hub Data (from https://utilitytransitionhub.rmi.org/data-download/ for Data Vault Prototype)

Copyright (C) 2021 OS-Climate

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

### We have a local copy rooted in the S3_BUCKET : RMI/data_download/*

### The next step is to enrich with OS-C Factor metadata

Contributed by Michael Tiemann (Github: MichaelTiemannOSC)

Load Credentials

In [ ]:
# From the AWS Account page, copy the export scripts from the appropriate role using the "Command Line or Programmatic Access" link
# Paste the copied text into ~/credentials.env

from dotenv import dotenv_values, load_dotenv
import os
import pathlib

dotenv_dir = os.environ.get('CREDENTIAL_DOTENV_DIR', os.environ.get('PWD', '/opt/app-root/src'))
dotenv_path = pathlib.Path(dotenv_dir) / 'credentials.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path,override=True)

Create an S3 resource for the bucket holding source data

In [ ]:
import boto3
s3_resource = boto3.resource(
    service_name="s3",
    endpoint_url=os.environ['S3_LANDING_ENDPOINT'],
    aws_access_key_id=os.environ['S3_LANDING_ACCESS_KEY'],
    aws_secret_access_key=os.environ['S3_LANDING_SECRET_KEY'],
)
bucket = s3_resource.Bucket(os.environ['S3_LANDING_BUCKET'])

rmi_folder = bucket.Object('RMI/data_download')

files = []
for file in bucket.objects.filter(Prefix='RMI/data_download'):
    if file.key.endswith('csv'):
        files.append(file.key)
files[0:]

Build a map and define schema mapping logic for parquet to sql

In [ ]:
_p2smap = {
    'object': 'varchar',
    'string': 'varchar',
    'str': 'varchar',
    'float32': 'real',
    'Float32': 'real',
    'float64': 'double',
    'Float64': 'double',
    'int32': 'integer',
    'Int32': 'integer',
    'int64': 'bigint',
    'Int64': 'bigint',
    'category': 'varchar',
    'datetime64[ns, UTC]': 'timestamp',
    'datetime64[ns]': 'timestamp'
}

def pandas_type_to_sql(pt):
    st = _p2smap.get(pt)
    if st is not None:
        return st
    raise ValueError("unexpected pandas column type '{pt}'".format(pt=pt))

# add ability to specify optional dict for specific fields?
# if column name is present, use specified value?
def generate_table_schema_pairs(df):
    ptypes = [str(e) for e in df.dtypes.to_list()]
    stypes = [pandas_type_to_sql(e) for e in ptypes]
    pz = list(zip(df.columns.to_list(), stypes))
    return ",\n".join(["    {n} {t}".format(n=e[0],t=e[1]) for e in pz])

In [ ]:
import trino

conn = trino.dbapi.connect(
    host=os.environ['TRINO_HOST'],
    port=int(os.environ['TRINO_PORT']),
    user=os.environ['TRINO_USER'],
    http_scheme='https',
    auth=trino.auth.JWTAuthentication(os.environ['TRINO_PASSWD']),
    verify=True,
)
cur = conn.cursor()

In [ ]:
# Show available schemas to ensure trino connection is set correctly
cur.execute('show schemas in osc_datacommons_dev')
cur.fetchall()

In [ ]:
schemaname = 'rmi_utility_transition_hub'
cur.execute('create schema if not exists osc_datacommons_dev.' + schemaname)
cur.fetchall()

Load RMI data file using pandas *read_csv* and appropriate dtype dictionaries

In [ ]:
import pandas as pd

dtype_dict = {
    # Table Name maps to dtypes (note that NaNs cannot encode to integers, so must do ex post facto fixes)
    'assets_earnings_investments':
        {'respondent_id':'int32', 'year':'int32',
         'asset_value':'float64', 'earnings_value':'float64', 'investment_value':'float64'},
    'customers_sales':
        {'respondent_id':'int32', 'year':'int32',
         'sales':'float64', 'revenues':'float64'},
    'debt_equity_returns':
        {'respondent_id':'int32', 'year':'int32',
         'rate_base_actual':'float64', 'equity_actual':'float64', 'debt_actual':'float64', 
         'equity_ratio_actual':'float64', 'returns_actual':'float64', 'earnings_actual':'float64',
         'interest_actual':'float64', 'fed_tax_expense_actual':'float64',
         'pre_tax_net_income_actual':'float64', 'ROR_actual':'float64', 'ROE_actual':'float64', 
         'interest_rate':'float64',
         'effective_fed_tax_rate':'float64', 'equity_authorized':'float64', 'debt_authorized':'float64',
         'returns_authorized':'float64', 'earnings_authorized':'float64', 'interest_authorized':'float64', 
         'interest_rate_authorized':'float64'},
    'emissions_targets':
        { # Dirty data prevents this up-front: 'year':'int32',
         'CO2_historical':'float64', 'CO2_target':'float64', 'CO2_target_all_years':'float64', 'CO2_1point5C':'float64',
         'generation_historical':'float64', 'generation_projected':'float64', 'generation_1point5C':'float64',
         'CO2_intensity_historical':'float64', 'CO2_intensity_target':'float64', 'CO2_intensity_target_all_years':'float64', 'CO2_intensity_1point5C':'float64'},
    'expenditure_bills_burden':
        {'respondent_id':'int32', 'year':'int32',
         'expenditure':'float64', 'bill':'float64', 'burden':'float64'},
    'housing_units_income':
        {'respondent_id':'int32', 'year':'int32',
         'housing_units':'float64', 'income':'float64'},
    'net_plant_balance':
        {'respondent_id':'int32', 'year':'int32',
         'original_cost':'float64', 'accum_depr':'float64', 'net_plant_balance':'float64',
         'ARC':'float64', 'ARC_accum_depr':'float64', 'net_ARC':'float64'},
    'operations_emissions_by_fuel':
        {'respondent_id':'int32', 'year':'int32',
         'latitude':'float64', 'longitude':'float64',
         # Dirty data prevents this up-front: 'operating_month':'int32', 'operating_year':'int32',
         'capacity':'float64', 'year_end_capacity':'float64', 'generation':'float64', 'potential_generation':'float64'},
    'operations_emissions_by_tech':
        {'respondent_id':'int32', 'year':'int32',
         'latitude':'float64', 'longitude':'float64',
         'capacity':'float64', 'year_end_capacity':'float64', 'generation':'float64', 'potential_generation':'float64'},
    'revenue_by_tech':
        {'respondent_id':'int32', 'year':'int32', 'latitude':'float64', 'longitude':'float64',
         'capacity':'float64', 'year_end_capacity':'float64', 'generation':'float64', 'potential_generation':'float64'},
    'state_targets': str,
    'utility_information': {'respondent_id':'int32'},
    'utility_state_map': {'respondent_id':'int32'}
}

fillna_dict = {
    'assets_earnings_investments':
        {'asset_value': 0, 'earnings_value': 0, 'investment_value': 0},
    'customer_sales':
        {'customers': 0, 'sales': 0, 'revenues': 0}
}

dropna_dict = {
    'emissions_targets': {'respondent_id':'int32'},
}

In [ ]:
tablename_to_df = {}

def create_trino_pipeline (s3, schemaname, tablename, df):
    global tablename_to_df
    df.to_parquet('/tmp/{sname}.{tname}.parquet'.format(sname=schemaname, tname=tablename), index=False)
    tablename_to_df[tablename] = df
    s3.upload_file(
        Bucket=os.environ['S3_DEV_BUCKET'],
        Key='trino/{sname}/{tname}/{tname}.parquet'.format(sname=schemaname, tname=tablename),
        Filename='/tmp/{sname}.{tname}.parquet'.format(sname=schemaname, tname=tablename)
    )
    cur.execute('.'.join(['drop table if exists osc_datacommons_dev', schemaname, tablename]))
    cur.fetchall()
    
    schema = generate_table_schema_pairs(df)

    tabledef = """create table if not exists osc_datacommons_dev.{sname}.{tname}(
{schema}
) with (
    format = 'parquet',
    external_location = 's3a://{bucket}/trino/{sname}/{tname}/'
)""".format(schema=schema,bucket=os.environ['S3_DEV_BUCKET'],sname=schemaname,tname=tablename)
    print(tabledef)

    # tables created externally may not show up immediately in cloud-beaver
    cur.execute(tabledef)
    cur.fetchall()

In [ ]:
# Create an S3 client
s3 = boto3.client(
    service_name="s3",
    endpoint_url=os.environ['S3_DEV_ENDPOINT'],
    aws_access_key_id=os.environ['S3_DEV_ACCESS_KEY'],
    aws_secret_access_key=os.environ['S3_DEV_SECRET_KEY'],
)

df_nn = None

for f in files:
    tablename = f.split('/')[-1].split('.')[0]
    print(f)
    if tablename=='state_utility_policies':
        df = pd.read_csv(bucket.Object(f).get()['Body'],
                         dtype={'respondent_id':'int32'},parse_dates=['date_updated'],dayfirst=True)
    else:
        df = pd.read_csv(bucket.Object(f).get()['Body'], dtype=dtype_dict[tablename])
    if tablename in dropna_dict:
        df.dropna(subset=list(dropna_dict[tablename].keys()), inplace=True)
    if tablename in fillna_dict:
        df.fillna(value=fillna_dict[tablename], inplace=True)
    
    # This code relies on seeing 'operations_emissions_by_fuel' before 'operations_emissions_by_tech'
    if tablename in ['operations_emissions_by_fuel', 'operations_emissions_by_tech']:
        # Both tables duplicate the 'Purchased Power' and 'EE & DR' data.
        # We only need one copy, which we create as 'other_generation'
        if tablename=='operations_emissions_by_fuel':
            df_anon = df[df['plant_name_eia'].isna()]
            # Drop many NULL columns we don't need
            df_anon.dropna(axis=1, how='all', inplace=True)
            create_trino_pipeline (s3, schemaname, 'other_generation', df_anon)
            df_nn = df[~df['operating_month'].isna()]
        df.dropna(subset=['plant_name_eia'], inplace=True)
        # For some reason, data before 2010 is sometimes note filled in.  This 
        for index, row in df[df['operating_month'].isna()].iterrows():
            # df_nn is only computed once, from 'operations_emissions_by_fuel'
            df0 = df_nn[(df_nn['respondent_id']==row['respondent_id']) & (df_nn['generator_id']==row['generator_id'])][['operating_month', 'operating_year']]
            if len(df0)==0:
                # In this case we have no prior data to refer to
                continue
            om, oy = df0.iloc[0]
            df.loc[index, ('operating_month', 'operating_year')] = om, oy
        for colname in ['operating_month', 'operating_year']:
            # df[colname] = pd.to_numeric(df[colname],downcast='int32')
            pass
    # if tablename in tidy_dict:
    #     tidy_df = df.melt(id_vars=tidy_dict[tablename][0], value_vars=tidy_dict[tablename][2],
    #                       var_name=tidy_dict[tablename][1][0], value_name=tidy_dict[tablename][1][1])
    #     tidy_df[tidy_dict[tablename][1][0]] = tidy_df[tidy_dict[tablename][1][0]].apply(lambda x: x.split('_')[0])
    #     tidy_df.dropna(subset=[tidy_dict[tablename][1][1]],inplace=True)
    create_trino_pipeline (s3, schemaname, tablename, df)

print('Done!')

## Load metadata following an ingestion process into trino metadata store

### The schema is *metastore*, and the table names are *meta_schema*, *meta_table*, *meta_field*

In [ ]:
metastore = 'metastore'

# Create a metadata schema with tables for the three layers of metadata: schema, table, and field.

meta_schema = 'meta_schema'
meta_table = 'meta_table'
meta_field = 'meta_field'

# These metadata tables are local to this ingestion process.
# We will insert/merge with master metadata tables later

metadata_to_df = {
    # For each data source there is a single entry in the _schema_table
    meta_schema: pd.DataFrame(data=[], columns=[]),
    # For each data source there are one or more tables in the _tables_table
    meta_table: pd.DataFrame(data=[],
                    columns=['tname', 'parent_schema', 'source', 'processing_pipeline']),
    # For each table there are one or more fields in the fields_table
    meta_field: pd.DataFrame(data=[],
                    columns=['fname', 'parent_table', 'type', 'dimension', 'description'])
}

cur.execute('create schema if not exists osc_datacommons_dev.' + metastore)
cur.fetchall()

Create the actual metadata for the source.  In this case, it is *rmi_utility_transition_hub*

We read and interpret the data dictionary that comes with the data sources.

In [ ]:
import io

rmi_dd_obj = None
for rmi_dd_obj in bucket.objects.filter(Prefix='RMI/data_download/RMI Utility Transition Hub Data Dictionary.xlsx'):
    break

rmi_dd_bytes = io.BytesIO(bucket.Object(rmi_dd_obj.key).get()['Body'].read())
rmi_dd = pd.read_excel(rmi_dd_bytes, sheet_name='Overview', dtype=str)
# Drop empty left column
rmi_dd.drop('Unnamed: 0', axis=1, inplace=True)
title = rmi_dd.loc[0, 'Unnamed: 1']
general_overview = rmi_dd.loc[1, 'Unnamed: 2']
scope = rmi_dd.loc[2, 'Unnamed: 2']
limitations_to_scope = rmi_dd.loc[3, 'Unnamed: 2']
# Drop non-table data captured above
rmi_dd.drop(list(range(0,5)), inplace=True)

In [ ]:
_mds_df = metadata_to_df[meta_schema]

In [ ]:
_mds_df['title'] = title
_mds_df['description'] = general_overview
_mds_df['version'] = 'Released in September 2021'
_mds_df['uri'] = 'https://utilitytransitionhub.rmi.org/data-download/'
_mds_df['copyright'] = '© 2021 RMI'
_mds_df['license'] = 'Creative Commons Attribution-Noncommercial 4.0 International Public License (CC BY-NC)'
_mds_df['contact'] = 'utilitytransitionhub@rmi.org'
_mds_df['abstract'] = '\n'.join([scope, limitations_to_scope])
_mds_df['name'] = 'rmi_utility_transition_hub'

In [ ]:
metadata_to_df[meta_schema] = _mds_df

Iterate through tablenames until we get to *Additional Information*, storing all the names and descriptions of the tables.

In [ ]:
_mdt_df = metadata_to_df[meta_table]
_mdt_df['tname'] = pd.Series(tablename_to_df.keys(), dtype=object)
_mdt_df['parent_schema'] = pd.Series([ meta_schema ] * len(_mdt_df['tname']), dtype=object)
_mdt_df['description'] = pd.Series([ '' ] * len(_mdt_df['tname']), dtype=object)

metadata_to_df[meta_table] = _mdt_df

for tablename, description in rmi_dd.itertuples(index=False):
    if tablename == 'Additional Information':
        break
    _mdt_df.loc[_mdt_df['tname']==tablename, 'description'] = description

# Add in this one table we generate ourselves from the _generation tables
_mdt_df.loc[_mdt_df['tname']=='other_generation'] = 'EE & DR as well as Purchased Power, all of which count toward `avoided generation`'

We've already collected the field names, types, dimensions, etc., of each table into our dataframes.
But there's additional metadata we can collect from the data dictionary

In [ ]:
_mdf_df = metadata_to_df[meta_field]

import math
units_dict = {
    '$': 'USD',
    'number': ''
}

for tablename, df in tablename_to_df.items():
    # Initialize field metadata from dataframes of tables we have written out
    if _mdf_df['fname'].empty==False:
        new_df = pd.DataFrame()
        new_df['fname'] = pd.Series(list(df.columns))
        new_df['parent_table'] = pd.Series([tablename] * len(df.columns))
        new_df['type'] = pd.Series([str(t) for t in df.dtypes])
        _mdf_df = _mdf_df.append(new_df, ignore_index=True)
    else:
        _mdf_df['fname'] = pd.Series(df.columns)
        _mdf_df['parent_table'] = pd.Series([tablename] * len(df.columns))
        _mdf_df['type'] = pd.Series([str(t) for t in df.dtypes])
    # Flesh out additional info from data dictionary
    if tablename == 'other_generation':
        # We have to enter this metadata by hand (later) for the tidy table we created
        continue
    dd_df = pd.read_excel(rmi_dd_bytes, sheet_name=tablename, dtype=str)
    # Drop empty left column
    dd_df.drop('Unnamed: 0', axis=1, inplace=True)
    dd_df.set_axis(['Data Field'] + list(dd_df.columns[1:-4]) + ['Definition', 'Units', 'Data Source', 'Methodology'], axis=1,inplace=True)
    # Data Field; (sub1; (sub2;)) Definition; Units; Data Source; Methodology
    for field, definition, units in dd_df[['Data Field', 'Definition', 'Units']].dropna(subset=['Data Field']).itertuples(index=False):
        _mdf_df.loc[(_mdf_df['parent_table']==tablename) & (_mdf_df['fname']==field), 'description'] = definition
        if type(units)==str:
            if units in units_dict:
                units = units_dict[units]
            _mdf_df.loc[(_mdf_df['parent_table']==tablename) & (_mdf_df['fname']==field), 'dimension'] = units

# Now add descriptions for *other_generation*
for field, description in _mdf_df.loc[_mdf_df['parent_table']=='operations_emissions_by_fuel', ['fname', 'description']].dropna(subset=['description']).itertuples(index=False):
    _mdf_df.loc[(_mdf_df['parent_table']=='other_generation') & (_mdf_df['fname']==field), 'description'] = description

metadata_to_df[meta_field] = _mdf_df

### The following is logically wrong because we're just slamming data into the metastore.  Next step is to update/insert data in case we are not the first/only ingestion process ever in the whole universe

In [ ]:
for tablename, df in metadata_to_df.items():
    schema = generate_table_schema_pairs(df)
    tabledef = """create table if not exists osc_datacommons_dev.{sname}.{tname}(
{schema}
) with (
    format = 'parquet',
    external_location = 's3a://{bucket}/trino/{sname}/{tname}/'
)""".format(schema=schema,bucket=os.environ['S3_DEV_BUCKET'],sname=metastore,tname=tablename)
    print(tabledef)
    
    cur.execute(tabledef)
    cur.fetchall()

In [ ]:
metadata_to_df[meta_field].dropna(subset=['dimension'])